### L'objectif de ce script est de charger les données en ligne de OpenFoodFacts pour les stocker dans un fichier décompressé Mongo DB Dump

## Panneau de configuration

In [11]:
import requests
import gzip
import pandas as pd
from io import BytesIO
import os
import shutil
from requests.exceptions import RequestException

In [12]:
url = "https://static.openfoodfacts.org/data/openfoodfacts-products.jsonl.gz"
fileNbr = '00'
projectPath = "/home/carolus/Documents/school/green_ia/" 
jsonGz = projectPath + "data/" + fileNbr + "_openfoodfacts" + ".jsonl.gz"
linesPerFile = 10000 # nombre de ligne pour chaque petit csv
csvOutput = projectPath + "data/" + fileNbr + "_openfoodfacts_00/" # dossier des minis csv

In [13]:
# fonction pour reprendre le téléchargement
def downloadFile(url, jsonGz):
    while True:
        try:
            # vérifier si fichier existe déjà et obtenir sa taille
            fileSize = 0
            if os.path.exists(jsonGz):
                fileSize = os.path.getsize(jsonGz)

            headers = {"range": f"bytes={fileSize}-"}
            response = requests.get(url, headers=headers, stream=True)

            if response.status_code in [200, 206]:
                mode = 'ab' if fileSize else 'wb'
                with open(jsonGz, mode) as file:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            file.write(chunk)
                print(f"fichier téléchargé et sauvegardé ici: {jsonGz}")
                break  # sortir boucle une fois téléchargement terminé

            else:
                print(f"erreur: {response.status_code}")
                break  # sortir boucle si erreur statut

        except RequestException as e:
            print(f"pause, reprise du téléchargement : {e}")

In [14]:
# décompresser du fichier jsonl
def unGzFile(jsonGz, fileNbr, projectPath):
    jsonl = projectPath + "data/" + fileNbr + '_openfoodfacts.jsonl'
    with gzip.open(jsonGz, 'rb') as f_in:
        with open(jsonl, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    print(f'fichier {jsonl} décompressé avec succès')
    return jsonl

In [15]:
# conversion en fichier csv
def convertToCsv(jsonl, fileNbr, projectPath):
    heavyCsv = projectPath + "data/" + fileNbr + '_openfoodfacts_00.csv'
    chunksize = 10000  
    chunkIter = pd.read_json(jsonl, lines=True, chunksize=chunksize)

    for i, chunk in enumerate(chunkIter):
        if i == 0:
            chunk.to_csv(heavyCsv, index=False, escapechar='\\')
        else:
            chunk.to_csv(heavyCsv, mode='a', header=False, index=False, escapechar='\\')

    print("conversion vers csv terminée")
    return heavyCsv

In [16]:
def deleteFile(filePath):
    if os.path.exists(filePath):
        os.remove(filePath)
        print(f"fichier {filePath} supprimé")
    else:
        print(f"erreur, fichier {filePath} n'existe pas")


In [17]:
def createFolder(folderPath):
    try:
        os.makedirs(folderPath, exist_ok=True)
        print(f"dossier créé avec succès: {folderPath}")
    except OSError as e:
        print(f"erreur création du dossier: {e}")

In [18]:
def splitCsv(csvFile, linesPerFile, csvOutput):
    try:
        chunkSize = linesPerFile
        chunks = pd.read_csv(csvFile, chunksize=chunkSize, on_bad_lines='skip')
        
        all_columns = set()
        for chunk in chunks:
            all_columns.update(chunk.columns)
        
        chunks = pd.read_csv(csvFile, chunksize=chunkSize, on_bad_lines='skip')
        
        for i, chunk in enumerate(chunks):
            for col in all_columns:
                if col not in chunk.columns:
                    chunk[col] = None
            chunk = chunk[list(all_columns)]
            
            outputFile = f"{csvOutput}{i+1}_openfoodfacts_00.csv"
            chunk.to_csv(outputFile, index=False)
            print(f"fichier {outputFile} sauvegardé avec {len(chunk)} lignes")
            
    except Exception as e:
        print(f"warning")

In [19]:
downloadFile(url, jsonGz)
jsonl = unGzFile(jsonGz, fileNbr, projectPath)
deleteFile(jsonGz)
heavyCsv = convertToCsv(jsonl, fileNbr, projectPath)
deleteFile(jsonl)
createFolder(csvOutput) 
splitCsv(heavyCsv, linesPerFile, csvOutput)

pause, reprise du téléchargement : ('Connection broken: IncompleteRead(1082162279 bytes read, 6700550722 more expected)', IncompleteRead(1082162279 bytes read, 6700550722 more expected))
fichier téléchargé et sauvegardé ici: /home/carolus/Documents/school/green_ia/data/00_openfoodfacts.jsonl.gz
fichier /home/carolus/Documents/school/green_ia/data/00_openfoodfacts.jsonl décompressé avec succès
fichier /home/carolus/Documents/school/green_ia/data/00_openfoodfacts.jsonl.gz supprimé
conversion vers csv terminée
fichier /home/carolus/Documents/school/green_ia/data/00_openfoodfacts.jsonl supprimé
dossier créé avec succès: /home/carolus/Documents/school/green_ia/data/00_openfoodfacts_00/


/tmp/ipykernel_65219/2723353045.py:7: DtypeWarning: Columns (3,23,27,90,96,97,106,136,139,142,143,148,149,150,154,157,158,160,161,162,163,164,166,169,171,172,173,178,182,183,189,190,191,192,193,195,196,205,211,214,216,217,219,226,228,230,234,236,242,243,246,251,254,255,256,257,258,259,260,261,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,304,305,307,308,309,310,311,312,313,314,315,316,317,318,319,332,334,335,339,342,343,344,345,346,347,348,349,351,352,354,355,357,358,359,360,361,362,363,364,365,366,367,370,376,377,378,379,380,383,384,385,387,388,390,396,397,403,404,405,406,407,408,409,410,411,412,413,415,416,417,418,422,423,424,425,426,427,428,429,430,431,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,488,489,490,491,492,49

fichier /home/carolus/Documents/school/green_ia/data/00_openfoodfacts_00/1_openfoodfacts_00.csv sauvegardé avec 10000 lignes


/tmp/ipykernel_65219/2723353045.py:12: DtypeWarning: Columns (51,183,188,189,194,196,201,225,226,244,246,248,249,252,255,256,257,263,266,268,269,280,288,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,312,313,314,315,316,317,319,320,321,323,328,332,333,334,337,339,340,341,344,345,346,347,348,349,351,352,355,366,367,368,369,370,371,373,375,376,377,378,380,381,382,383,384,385,386,387,388,389,390,391,392,394,396,397,399,400,401,403,404,405,406,407,410,411,412,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,441,442,443,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,520,521,522,523,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,549,551,552,553

fichier /home/carolus/Documents/school/green_ia/data/00_openfoodfacts_00/2_openfoodfacts_00.csv sauvegardé avec 10000 lignes


...

fichier /home/carolus/Documents/school/green_ia/data/00_openfoodfacts_00/329_openfoodfacts_00.csv sauvegardé avec 10000 lignes


/tmp/ipykernel_65219/2723353045.py:12: DtypeWarning: Columns (10,42,47,58,62,70,79,86,105,106,108,112,140,167,175,176,185,192,197,199,200,209,211,212,213,214,215,216,219,223,224,228,229,230,232,234,235,238,239,240,241,242,243,244,245,246,247,248,250,252,253,254,255,257,258,259,260,261,262,263,265,266,267,268,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,356,359,360,361,362,363,364,366,367,368,371,372,373,374,375,376,377,378,380,381,382,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,405,406,407,408,409,411,412,413,414,415,417,419,422,423,424,425,426,428,429,431,434,435,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,458,459,461,462,463,464,465,467,468,469,470,47

fichier /home/carolus/Documents/school/green_ia/data/00_openfoodfacts_00/330_openfoodfacts_00.csv sauvegardé avec 10000 lignes


/tmp/ipykernel_65219/2723353045.py:12: DtypeWarning: Columns (3,5,6,18,31,33,49,51,55,56,61,66,96,98,111,163,170,172,175,180,182,186,189,193,194,195,196,198,204,205,208,209,210,214,218,219,220,221,222,223,224,225,226,227,228,229,232,233,234,235,236,237,238,239,240,242,243,250,251,252,255,256,257,258,259,260,262,263,264,265,266,268,269,270,271,272,273,274,276,278,279,280,282,283,287,288,289,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,308,309,310,311,312,313,314,316,317,318,319,320,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,401,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,420,422,423,426,427,428,429,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453

fichier /home/carolus/Documents/school/green_ia/data/00_openfoodfacts_00/331_openfoodfacts_00.csv sauvegardé avec 5361 lignes
